# Manually pipelining tasks: Neuroimaging example

## Summary

In this *Notebook* you'll be able to execute two subsequent operations on anatomical T<sub>1</sub>-weighted MRI images of adult human heads.

The first typical operation is *brain extraction*, which involves identifying brain tissue and cerebrospinal fluid (CSF) in the image, and discard non-brain tissue (e.g. skull, neck, skin and fat, etc.)

Only once the brain has been *extracted*, we can use machine learning to identify three major types of tissue: gray matter (GM), white matter and CSF.

## Setting up

First, we import `Path` from *Python*'s standard library, and the `NiiVue` class from the *ipyniivue* library for visualization of 3D brain images.

Second, we define some paths (`DATA_PATH` and `OUTPUTS_PATH`) where we will be finding and storing data (respectively).

In [ ]:
from pathlib import Path
from ipyniivue import NiiVue

In [ ]:
DATA_PATH = Path.home() / "data" / "ds000005"
OUTPUTS_PATH = Path.home() / "outputs"

## Visualizing a T1w image of the human brain

The interactive viewer allows for navigation of three plane cuts through the volume, as well as a 3D rendering.

In [ ]:
nv = NiiVue()
nv.load_volumes([{"path": DATA_PATH / "sub-01/anat/sub-01_T1w.nii.gz"}])
nv

## Running a brain extraction program

We will extract the brain using the `3dSkullStrip` executable of the [AFNI](https://afni.nimh.nih.gov/) open-source software library.

We can execute bash commands from within the Jupyter Notebook (that is, execute code that *is not* Python code) by preceding command lines by the exclamation mark symbol `!`, so, in this case, we write `!3dSkulStrip` to tell this notebook that the cell contains code to execute on a terminal shell.

As an extra exercise, create a new cell and execute the following command to read the help of `3dSkullStrip`:

```Bash
!3dSkullStrip -help
```

In [ ]:
!3dSkullStrip -input $HOME/data/ds000005/sub-01/anat/sub-01_T1w.nii.gz -prefix $HOME/outputs/sub-01_desc-brain_T1w.nii.gz

## Visualizing the output

We can now look at the result of executing `3dSkullStrip` on a head image.

In [ ]:
nv = NiiVue()
nv.load_volumes([{"path": OUTPUTS_PATH / "sub-01_desc-brain_T1w.nii.gz"}])
nv

## Identifying brain tissues

We will use now another program within AFNI's distribution: `3dSeg`.

By default, the program will try to identify GM, WM and CSF.
The output will be written in a custom format of AFNI, so we add a third task in our pipeline to convert it into a more standard format (called NIfTI). This is achieved with the command `3dAFNItoNIFTI`.

In [ ]:
!3dSeg -anat $HOME/outputs/sub-01_desc-brain_T1w.nii.gz -mask AUTO -prefix $HOME/outputs/segments

In [ ]:
!3dAFNItoNIFTI -prefix $HOME/outputs/sub-01_seg-tissues_dseg.nii.gz $HOME/outputs/segments/Classes+orig

## Visualizing the separated brain tissues

Now we can load the output after conversion and we can see how the segmentation performed.

In [ ]:
nv = NiiVue()
nv.load_volumes([{"path": OUTPUTS_PATH / "sub-01_seg-tissues_dseg.nii.gz", "colormap": "nih"}])
nv